In [27]:
import pandas as pd
import numpy as np

# Adatok beolvasása

In [28]:
df = pd.read_csv("training.csv")

# Data preparation

In [29]:
from scipy import stats

In [30]:
#It is not worth it everywhere
def categorial_without_missing_handling(df, column):
    df = pd.get_dummies(df, columns=[column])
    return df

def categorial_with_missing_handling(df, column):
    df[column + '_missing'] = df[column].isnull()
    df[column] = df[column].fillna(df[column].mode()[0])
    df = pd.get_dummies(df, columns=[column])
    return df

def categorial_values(df):
    df = categorial_without_missing_handling(df, "city")
    df = categorial_with_missing_handling(df, "property_subtype")
    df = categorial_without_missing_handling(df, "view_type")
    df = categorial_with_missing_handling(df, "orientation")
    df = categorial_without_missing_handling(df, "garden_access")
    df = categorial_without_missing_handling(df, "heating_type")
    df = categorial_with_missing_handling(df, "elevator_type")
    
    #Did not helped
    #df.loc[df.property_condition_type != 'missing_info', "property_condition_type_missing_info"] = 0
    #df.loc[df.property_condition_type == 'missing_info', "property_condition_type_missing_info"] = 1
    #df.loc[df.property_condition_type == 'missing_info', "property_condition_type"] = "good"
    df = categorial_without_missing_handling(df, "property_condition_type")
    
    
    return df

In [31]:
def clear_non_scalar_value(df, property, value, replace):
    df_clean = df.loc[df[property] == value, property] = replace
    return df

def fill_missing_numeric_values(df):
    df["small_room_cnt"] = df["small_room_cnt"].fillna(0)

    df["balcony_area"] = df["balcony_area"].fillna(0)

    df["building_floor_count"] = df["building_floor_count"].fillna(0)
    df = clear_non_scalar_value(df, "building_floor_count", 'more than 10', 11)

    df["property_floor"] = df["property_floor"].fillna(0)
    df = clear_non_scalar_value(df, 'property_floor', 'ground floor', 0)
    df = clear_non_scalar_value(df, 'property_floor', 'mezzanine floor', 0.5)
    df = clear_non_scalar_value(df, 'property_floor', 'basement', -1.0)
    df = clear_non_scalar_value(df, 'property_floor', '10 plus', 11)

    return df

In [32]:
def create_new_datas(df):
    df['sm_price'] = df['price_created_at'] / df['property_area']
    df = df.replace([np.inf, -np.inf], np.nan)
    df['sm_price'] = df['sm_price'].fillna(0)
    
    
    
    return df

In [33]:
# None of this helped
def cut_irreal_values(df):
    #df['price_created_at'] = np.clip(df['price_created_at'],1,1000)
    #df['balcony_area'] = np.clip(df['balcony_area'],0,25)
    #df['room_cnt'] = np.clip(df['room_cnt'],0,8)
    #df['small_room_cnt'] = np.clip(df['small_room_cnt'],0,6)
    
    return df

In [34]:
def delete_unimportant_data(df):
    df = df.drop("county", axis=1)
    df = df.drop("property_type", axis=1)
    df = df.drop("postcode", axis=1)
    return df

In [35]:
def clear_dataframe(df):
    df = fill_missing_numeric_values(df)
    df = categorial_values(df)
    df = create_new_datas(df)
    #df = cut_irreal_values(df)
    df = delete_unimportant_data(df)
    
    return df

In [36]:
df = clear_dataframe(df)

### Choose features

In [37]:
input_features = df.columns.tolist()
input_features.remove('ad_view_cnt')
input_features.remove('id')

target = 'ad_view_cnt'

# Regression

### Train-test split

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [39]:
# TODO: STRATIFY
train, test = train_test_split(df, train_size=0.8, test_size=0.2, random_state=42)

#skf = StratifiedKFold(n_splits=5, random_state=242)
#for train_idx, test_idx in skf.split(df, df.ad_view_cnt):
#    X_train, y_train = df.loc[train_idx, input_features], df.loc[train_idx, target]
#    X_test, y_test = df.loc[test_idx, input_features], df.loc[test_idx, target]
#    print(
#        "{:5d} {:5d} {:5d} {:5d} {:.4f} {:.4f} {:1d}".format(
#            X_train.shape[0], X_test.shape[0], 
#            y_train.shape[0], y_test.shape[0], 
#            y_train.mean(), y_test.mean(),
#            len(set(y_train.index) & set(y_test.index))
#        )
#    )

X_train = train[input_features]
y_train = train[target]
X_test = test[input_features]
y_test = test[target]

### Standardization

In [40]:
from sklearn.preprocessing import StandardScaler

In [41]:
std_scale = StandardScaler().fit(X_train)
X_train = std_scale.transform(X_train)
X_test = std_scale.transform(X_test)

### Random Forest

In [42]:
#from sklearn.ensemble import RandomForestRegressor

In [43]:
#rf = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=431)

In [44]:
#rf = rf.fit(X_train, y_train)

In [45]:
#yhat_test = pd.Series(rf.predict(X_test), index=y_test.index)

### Gradient Boosting

In [46]:
from sklearn.ensemble import GradientBoostingRegressor

In [47]:
gb = GradientBoostingRegressor(n_estimators=900, learning_rate=0.01, max_depth=11, random_state=431)

In [48]:
gb = gb.fit(X_train, y_train)

In [49]:
yhat_test = pd.Series(gb.predict(X_test), index=y_test.index)

# Kiértékelés

### Testing RMSE

### Final test and export

In [50]:
#Reading the input
df_train = pd.read_csv("training.csv")
df_test = pd.read_csv("testing.csv")

#Data cleaning
df_train = clear_dataframe(df_train)
df_test = clear_dataframe(df_test)

#Features
X_train = df_train[input_features]
y_train = df_train[target]
X_test = df_test[input_features]

#Standardization
std_scale = StandardScaler().fit(X_train)
X_train = std_scale.transform(X_train)
X_test = std_scale.transform(X_test)

gb = gb.fit(X_train, y_train)
final_result = pd.DataFrame(gb.predict(X_test), columns=['target'])

In [51]:
final_result = np.clip(final_result,0,1000000)

In [52]:
final_result.insert(0, 'id', df_test.id)

In [53]:
final_result.to_csv('f2avxh.csv', index=False)